In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import lmfit as lm
from lmfit import Model

In [107]:
#raw
eV = pd.read_csv("/Users/cameronrichardson/Documents/School/Thesis/Code/data/~$S1102C15N01.01a")


eV = pd.DataFrame(eV)
eV_c = eV.loc[2:len(eV)]

eV_c.rename(columns={' Cell W (g)': 'volAdded'}, inplace=True) 
eV_c.rename(columns={' Cell T (oC)': 'mV'}, inplace=True)

vol = eV_c['volAdded']
mV = eV_c['mV']


eV.head()



/var/folders/rp/6325jg2164gg259yvxgz1lzm0000gn/T/ipykernel_30401/4167970281.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eV_c.rename(columns={' Cell W (g)': 'volAdded'}, inplace=True)
/var/folders/rp/6325jg2164gg259yvxgz1lzm0000gn/T/ipykernel_30401/4167970281.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eV_c.rename(columns={' Cell T (oC)': 'mV'}, inplace=True)


,Acid m (mol/kg),Cell W (g),Cell T (oC)
0,0.1,207.4,19.0
1,Salinity = 35.000,NaN,NaN
2,0,0.0,-129.9
3,1,0.1,-128.2
4,2,0.2,-125.8


In [40]:
#def cnst(Bt, St, kb, kw, k1, k2, a, b, c0, d0, rho_sw, I, sal, E):

#Solution matrix
matrix = pd.read_csv("/Users/cameronrichardson/Documents/School/Thesis/Code/data/NLLS_matrix.csv")

T = matrix.temp[0]+273.15
t = matrix.temp[0]
Na = matrix.molality[0]
K =matrix.molality[1]
Mg = matrix.molality[2]
Ca = matrix.molality[3]
Cl = matrix.molality[4]
SO42 = matrix.molality[5]
CO32 = matrix.molality[6]
B = matrix.molality[7]
DIC = matrix.molality[8]
TA = matrix.molality[9]


#Ionic strength 
I =  1/2*(Na + K + Mg*4 + Ca*4 + Cl + SO42*4 + CO32*4)

#Salinity
sal = I/19.92*(1000-1.0049)

#EQUIVALENT MOLALITY OF THE SOLUTION
E = 1/2*(Na + K + Mg*2 + Ca*2 + Cl + SO42*2 + CO32*2)


##########################################Constants##########################################################
#UPPSTROM, 1974  Total boron BT    mol/kg soln*/
Bt=0.000232/10.81*(sal/1.80655)

#MORRIS AND RILEY, 1966  Total sulfate ST   mol/kg soln*/
St=0.1400/96.09*(sal/1.80655)

#DICKSON,1990 Dissociation of bisultate ion = mol/kg soln	  */
ks = np.exp((-4276.1/T + 141.328 - 23.093*np.log(T) + (-13856/T +324.57 - 47.986*np.log(T))*np.sqrt(I) + (35474/T -771.54 + 114.723*np.log(T))*I -2698/T*I*np.sqrt(I) + 1776/T*I*I) + np.log(1-sal*0.001005))

#DICKSON 1990 KB  = mol/ kg soln , [H]sws	*/
kb = np.exp((-8966.90 - 2890.53*np.sqrt(sal) - 77.942*sal +1.728*sal*np.sqrt(sal) - 0.0996*sal*sal)/T +(148.0248 + 137.1942*np.sqrt(sal) + 1.62142*sal) +(-24.4344 - 25.085*np.sqrt(sal) - 0.2474*sal)*np.log(T) +0.053105*np.sqrt(sal)*T +np.log((1 + St/ks)/(1 + St/ks)))

#MILLERO, 1994	KWater	mol/kg soln*/
kw = np.exp(-13847.26/T + 148.96502 - 23.6521*np.log(T) +(118.67/T - 5.977 + 1.0495 *np.log(T))*np.sqrt(sal) -0.01615*sal)

#ROY et al 1993  K1  mol/kg soln,REFIT FROM MILLERO 1994 [H]sws*/
k1=np.exp(3.17537 - 2329.1378/T - 1.597015*np.log(T) + (-0.210502- 5.79495/T)*np.sqrt(sal) + 0.0872208*sal -0.00684651*sal*np.sqrt(sal))

# Mehrbach et al 1973 K2  mol/kg soln, REFIT FROM DICKSON-MILLERO 1987 [H]sws */
k2=np.exp(-(1394.7/T + 4.777 - 0.0184*sal + 0.000118*sal*sal)*np.log(10))
################################################################################

#THEORETICAL NERNST SLOPE RT/F  R=J/mol K  and F=C/mol*/
T_NS=np.log(10)*8.31441*T/96484.56

#DENSITY OF SEAWATER MILLERO AND POISSON, 1981           
a = 8.24493e-4 - 4.0899e-6*(t) + 7.6438e-8*t*t - 8.2467e-10*t*t*t+ 5.3875e-12*t*t*t*t
b = -5.72466e-6 + 1.0227e-7*t - 1.6546e-9*t*t
c0 = 4.8314e-7
d0 = 0.999842594 + 6.793953e-5*t - 9.09529e-6*t*t + 1.001685e-7*t*t*t- 1.120083e-9*t*t*t*t + 6.536332e-12*t*t*t*t*t

rho_sw = d0 + a*sal + b*sal*np.sqrt(sal) + c0*sal*sal #density g/cm3


#CALCULATION OF THE DEBYE-HUCKEL LIMITING SLOPE (MOELLER, 1988)
DHL = np.sqrt(I) / (1 + 1.2 * np.sqrt(I)) + (2 / 1.2) * np.log(1 + 1.2 * np.sqrt(I))
DHL = DHL * -(0.33690153 - 0.00063210043 * T + 9.14252359 / T - 0.0135143986 * np.log(T) + 0.00226089488 / (np.log(T) - 263) + 1.921185973E-06 * T * T + 45.2586464 / (680 - T))



print(matrix)

       species  molality  temp
0           Na  0.467147  19.0
1            K  0.010248   NaN
2           Mg  0.052836   NaN
3           Ca  0.010366   NaN
4           Cl  0.547159   NaN
5          SO4  0.028319   NaN
6          CO3  0.001105   NaN
7  Total Boron  0.000417   NaN
8          DIC  0.001105   NaN
9           TA  0.002209   NaN


In [8]:
# PARAMETERS FOR NaCl (MOLLER, 1988; GREENBERG AND MOLLER, 1989)
B0NACL = 14.3783204 + 0.00560767406 * T - 422.185236 / T - 2.51226677 * np.log(T) - 2.61718135E-06 * T * T + 4.43854508 / (680 - T) - 1.70502337 / (T - 227)
B1NACL = -0.483060685 + 0.00140677479 * T + 119.311989 / T - 4.23433299 / (T - 227)
CNACL = -0.100588714 - 1.80529413E-05 * T+ 8.61185543 / T + 0.012488095 * np.log(T) + 3.41172108E-08 * T * T + 0.0683040995 / (680 - T) + 0.293922611 / (T - 227)


# PARAMETERS FOR Na2SO4 (HOVEY ET AL., 1992)

B0NA2SO4 = 0.006536438 - 30.197349 * (1 / T - 1 / 298.15) - 0.20084955 * np.log(T / 298.15)
B1NA2SO4 = 0.8742642 - 70.014123 * (1 / T - 1 / 298.15) + 0.2962095 * np.log(T / 298.15)
CNA2SO4 = 0.007693706 + 4.5879201 * (1 / T - 1 / 298.15) + 0.019471746 * np.log(T / 298.15)

# PARAMETERS FOR NaHSO4 (CAMPBELL ET AL., 1993)

B0NAHSO4 = 0.0670967 - 0.00048949 * (T - 298.15) + 8.8426E-05 * (T - 298.15) **2
B1NAHSO4 = 0.3826401 - 0.018431 * (T - 298.15)
CNAHSO4 = -0.0039056

# PARAMETERS FOR NaHSO4 (PIERROT AND MILLERO, 1996)

B0NAHSO4 = 0.0544 - 0.0018478 * (T - 298.15) + 5.3937E-05 * (T - 298.15) **2
B1NAHSO4 = 0.3826401 - 0.018431 * (T - 298.15)
CNAHSO4 = -0.0039056

# PARAMETERS FOR NaHCO3 (PEIPER AND PITZER, 1982)
B0NAHCO3 = 0.028 + 0.001 * (T - 298.15) + 5.082001E-21 * (T - 298.15) **2
B1NAHCO3 = 0.044 + 0.0011 * (T - 298.15) - 3.88E-21 * (T - 298.15) **2
CNAHCO3 = 0

# PARAMETERS FOR NaBr (PITZER AND MAYORGA, 1973)
# B0NABR = 0.0973 + 0.0007692 * (T - 298.15)
# B1NABR = 0.2791 + 0.001079 * (T - 298.15)
# CNABR = 0.00116 - 9.3E-05 * (T - 298.15)

# PARAMETERS FOR Na2CO3 (PEIPER AND PITZER,1982)
B0NA2CO3 = 0.0362 + 0.00179 * (T - 298.15) + 1.694E-21 * (T - 298.15) **2
B1NA2CO3 = 1.51 + 0.00205 * (T - 298.15) + 1.626E-19 * (T - 298.15) **2
CNA2CO3 = 0.0052

# PARAMETERS FOR NaB(OH)4 (SIMONSON ET AL., 1987)
B0NABOH4 = -0.051 + 0.005264 * (T - 298.15)
B1NABOH4 = 0.0961 - 0.01068 * (T - 298.15)
CNABOH4 = 0.01498 - 0.00157 * (T - 298.15)

# # PARAMETERS FOR NaF (PITZER AND MAYORGA, 1973)
# B0NAF = 0.0215 + 0.0005361 * (T - 298.15)
# B1NAF = 0.2107 + 0.00087 * (T - 298.15)
# CNAF = 0

# PARAMETERS FOR NaHS (HERSHEY ET AL., 1988)
B0NAHS = 0.366 - 67.5 / T
B1NAHS = 0
CNAHS = -0.0127

# PARAMETERS FOR NaOH (0 to 350oC)(PABALAN AND PITZER, 1987)
B0NAOH = 276.82478 - 0.0028131778 + (-7375.5443 + 0.3701254) / T - 49.35997 * np.log(T) + (0.10945106 + 7.1788733E-06) * T + (-4.0218506E-05 - 5.8847404E-09) * T **2 + 11.931122 / (T - 227) + (2.4824963 - 0.004821741) / (647 - T)
B1NAOH = 462.86977 + (-10294.18) / T - 85.96058 * np.log(T) + 0.23905969 * T + (-0.00010795894) * T **2
CNAOH = -16.686897 + 0.00040534778 + (453.64961 - 0.051714017) / T + 2.9680772 * np.log(T) + (-0.0065161667 - 1.05530373E-06) * T + (2.376578E-06 + 8.9893405E-10) * T **2 - 0.68923899 / (T - 227) - 0.081156286 / (647 - T)

 # PARAMETERS FOR CaCl2 (MOLLER, 1988; GREENBERG AND MOLLER, 1989)
B0CACL2 = -94.1895832 - 0.0404750026 * T + 2345.50368 / T + 17.09123 * np.log(T) - 0.922885841 / (T - 263) + 1.51488122E-05 * T * T - 1.39082 / (680 - T)
B1CACL2 = 3.4787 - 0.015417 * T + 3.1791E-05 * T * T
CCACL2 = 19.3056024 + 0.00977090932 * T - 428.383748 / T - 3.57996343 * np.log(T) + 0.0882068538 / (T - 263) - 4.62270238E-06 * T * T + 9.91113465 / (680 - T)

# PARAMETERS FOR Ca2SO4 (MOLLER, 1988; GREENBERG AND MOLLER, 1989)

B0CASO4 = 0.15
B1CASO4 = 3
B2CASO4 = -129.399287 + 0.400431027 * T
CCASO4 = 0

 # PARAMETERS FOR Ca[mB(OH)4]2 (HERSHEY ET AL., 1986;SIMONSON ET AL,1988)

B0CABOH42 = -0.4462 + 0.005393 * (T - 298.15)
B1CABOH42 = -0.868 - 0.0182 * (T - 298.15)
B2CABOH42 = -15.88 - 0.002858 * (T - 303.15)**2
CCABOH42 = 0

# PARAMETERS FOR CaHCO3 (HARVIE, MOELLER, AND WEARE, 1983)
B0CAHCO32 = 0.4
B1CAHCO32 = 2.977
CCAHCO32 = 0

# PARAMETERS FOR Ca(HS)2 (HERSHEY ET AL., 1988)

B0CAHS2 = -0.105
B1CAHS2 = 3.43
CCAHS2 = 0

# PARAMETERS FOR CaHSO4 (HARVIE, MOELLER, AND WEARE, 1983)

B0CAHSO4 = 0.2145
B1CAHSO4 = 2.53
CCAHSO4 = 0

# PARAMETERS FOR CaBr2 (PITZER AND MAYORGA, 1973)

# B0CABR2 = 0.3816 - (3 / 4) * 0.000697 * (T - 298.15)
# B1CABR2 = 1.61325 + (3 / 4) * 0.00805 * (T - 298.15)
# CCABR2 = -0.002572

# PARAMETERS FOR KCl (GREENBERG AND MOLLER, 1989)

B0KCL = 26.7375563 + 0.010072105 * T - 758.485453 / T - 4.70624175 * np.log(T) - 3.75994338E-06 * T * T
B1KCL = -7.4155962 + 322.892989 / T + 1.16438557 * np.log(T) - 5.9457814 / (T - 227)
CKCL = -3.30531334 - 0.00129807848 * T + 91.27121 / T + 0.58645018 * np.log(T) + 4.95713573E-07 * T * T

# PARAMETERS FOR K2SO4 (GREENBERG AND MOLLER, 1989)

B0K2SO4 = 40.7908797 + 0.00826906675 * T - 1418.42998 / T - 6.74728848 * np.log(T)
B1K2SO4 = -13.1669651 + 0.023579323 * T + 2067.12594 / T
CK2SO4 = -0.0188

# PARAMETERS FOR KHSO4 (HARVIE MOLLER, AND WEARE, 1983)

B0KHSO4 = -0.0003
B1KHSO4 = 0.1735
CKHSO4 = 0

# PARAMETERS FOR KHCO3 (ROY ET AL., 1983)

B0KHCO3 = -0.0107 + 0.001 * (T - 298.15)
B1KHCO3 = 0.0478 + 0.0011 * (T - 298.15) + 6.776E-21 * (T - 298.15) **2
CKHCO3 = 0

# PARAMETERS FOR K2CO3 (SIMONSON ET AL., 1987)
B0K2CO3 = 0.1288 + 0.0011 * (T - 298.15) - 5.1E-06 * (T - 298.15) **2
B1K2CO3 = 1.433 + 0.00436 * (T - 298.15) + 2.07E-05 * (T - 298.15) **2
CK2CO3 = 0.0005

# PARAMETERS FOR KB(OH)4 (SIMONSON ET AL., 1987)

B0KBOH4 = 0.1469 + 0.002881 * (T - 298.15)
B1KBOH4 = -0.0989 - 0.006876 * (T - 298.15)
CKBOH4 = -0.05643 - 0.000956 * (T - 298.15)

# PARAMETERS FOR KBr (PITZER AND MAYORGA, 1973)

# B0KBR = 0.0569
# B1KBR = 0.2122
# CKBR = -0.0018

# PARAMETERS FOR KHS (5 TO 25oC) (MILLERO,1995; HERSHEY ET AL.,1988)

B0KHS = 1.017 - 251.04 / T
B1KHS = 0
CKHS = -0.033

# PARAMETERS FOR KOH (PITZER AND MAYORGA, 1973)

B0KOH = 0.1298
B1KOH = 0.32
CKOH = 0.0041

# PARAMETERS FOR KF (PITZER AND MAYORGA, 1973)

# B0KF = 0.08089 + 0.000214 * (T - 298.15)
# B1KF = 0.2021 + 0.000544 * (T - 298.15)
# CKF = 0.00093 - 5.95E-05 * (T - 298.15)

# PARAMETERS FOR HCl (CAMPBELL ET AL., 1993)

B0HCL = 1.2859 - 0.0021197 * T - 142.5877 / T
B1HCL = -4.4474 + 0.008425698 * T + 665.7882 / T
CHCL = -0.305156 + 0.000516 * T + 45.52154 / T

# PARAMETERS FOR H,HSO4 (HOVEY ET AL., 1993)

B0HHSO4 = 0.1832845
B1HHSO4 = 0.6423595
CHHHSO4 = 0.0048954

# PARAMETERS FOR H,SO4 (HARVIE, MOLLER, AND WEARE, 1983)

B0HSO4 = 0.1990501
B1HSO4 = -0.271211
CHSO4 = 0.0011043

# PARAMETERS FOR HBr (PITZER AND MAYORGA, 1973)
# B0HBR = 0.196 - 0.0002049 * (T - 298.15)
# B1HBR = 0.3564 + 0.0004467 * (T - 298.15)
# CHBR = 0.00827 - 5.685E-05 * (T - 298.15)

# PARAMETERS FOR MgCl2 (PABALAN AND PITZER,1987)
B0MGCL2 = 0.57606 - 0.000931654 * T + 5.93915E-07 * (T**2)
B1MGCL2 = 2.60135 - 0.0109438 * T + 2.60169E-05 * (T**2)
CMGCL2 = 0.059532 - 0.000249949 * T + 2.41831E-07 * (T**2)

# PARAMETERS FOR MgSO4 (PABALAN AND PITZER, 1987)
B0MGSO4 = -1.0282 * (T / 2 + 298 **2 / (2 * T) - 298) + 0.008479 * (T **2 / 6 + 298**3 / (3 * T) - 298 **2 / 2) - 2.33667E-05 * (T **3 / 12 + 298 **4 / (4 * T) - 298 **3 / 3) + 2.1575E-08 * (T **4 / 20 + 298 **5 / (5 * T) - 298 **4 / 4)+ (298 - 298 **2 / T) * 0.00068402 + 0.21499
B1MGSO4 = -0.29596 * (T / 2 + 298 **2 / (2 * T) - 298) + 0.00094564 * (T **2 / 6 + 298 **3 / (3 * T) - 298 **2 / 2) + (298 - 298 **2 / T) * 0.011028 + 3.3646
B2MGSO4 = -13.764 * (T / 2 + 298 **2 / (2 * T) - 298) + 0.12121 * (T **2 / 6 + 298 **3 / (3 * T) - 298 **2 / 2) - 0.00027642 * (T **3 / 12 + 298 **4 / (4 * T) - 298 **3 / 3) - 0.21515 * (298 - 298 **2 / T) - 32.743
CMGSO4 = 0.10541 * (T / 2 + 298 **2 / (2 * T) - 298) - 0.00089316 * (T **2 / 6 + 298 **3 / (3 * T) - 298 **2 / 2) + 2.51E-06 * (T **3 / 12 + 298 **4 / (4 * T) - 298 **3 / 3) - 2.3436E-09 * (T **4 / 20 + 298 **5 / (5 * T) - 298 **4 / 4) - 8.7899E-05 * (298 - 298 **2 / T) + 0.006993

#PARAMETERS FOR MgHSO4 (CAMPBELL ET AL., 1993)
GHSO4 = -9.52
B1MGHSO4 = 24.3
CMGHSO4 = 12.12

# PARAMETERS FOR MgHSO4 (PIERROT AND MILLERO, 1996)
B0MGHSO4 = 0.02212
B1MGHSO4 = 5.02348
CMGHSO4 = 0.16699

# PARAMETERS FOR MgHCO3 (PITZER ET AL., 1985)

B0MGHCO32 = 0.03
B1MGHCO32 = 0.8
CMGHCO32 = 0

# PARAMETERS FOR Mg[mB(OH)4]2 (HERSHEY ET AL, 1986; SIMONSON ET AL., 1988)

B0MGBOH42 = -0.623 + 0.006496 * (T - 298.15)
B1MGBOH42 = 0.2515 - 0.01713 * (T - 298.15)
B2MGBOH42 = -11.47 - 0.00324 * (T - 303.15) **2
CMGBOH42 = 0

# PARAMETERS FOR MgBr2 (PITZER AND MAYORGA, 1973)

# B0MGBR2 = 0.432675 - (3 / 4) * 7.5E-05 * (T - 298.15)
# B1MGBR2 = 1.75275 + (3 / 4) * 0.00515 * (T - 298.15)
# CMGBR2 = 0.003124

 # PARAMETERS FOR Mg(HS)2 (HERSHEY ET AL., 1988)

B0MGHS2 = 0.17
B1MGHS2 = 2.78
CMGHS2 = 0

# PARAMETERS FOR SrCl2 (PITZER AND MAYORGA, 1973)

# B0SRCL2 = 0.28575 + (3 / 4) * 0.000956 * (T - 298.15)
# B1SRCL2 = 1.66725 + (3 / 4) * 0.00379 * (T - 298.15)
# CSRCL2 = -0.0013

# PARAMETERS FOR SrBr2 (PITZER AND MAYORGA, 1973)
B0SRBR2 = 0.33113 + (3 / 4) * 0.000437 * (T - 298.15)
B1SRBR2 = 1.7115 + (3 / 4) * 0.00871 * (T - 298.15)
CSRBR2 = 0.00123

# PARAMETERS FOR SrSO4 (EQUAL TO CASO4)

# B0SRSO4 = B0CASO4
# B1SRSO4 = B1CASO4
# B2SRSO4 = B2CASO4
# CSRSO4 = CCASO4


In [ ]:
 #CONVERT FROM VOLUME TO MASS UNITS AND FROM mVOLTS TO VOLTS      


# vol[i]=vol[i]*(consts[11]);
# emf[i]=emf[i]/1000
# ig[i]=.1/1000
# E0=cellE0/1000
# CALCULATE [H+] USING INITIAL ESTIMATE OF E0 AND NERNST SLOPE    

# h[i]=exp(-((E0 - emf[i])/consts[1])*np.log(10))


# np.exp(-((E0 - mV[0:i]) / T_NS) * np.log(10))


np.exp(-((375 - mV[0:1]) / T_NS) * np.log(10))


np.float64(0.057968918137848366)

In [ ]:
#DENSITY OF HCL ASSUMING TOTAL IONIC STRENGTH OF HCL AND NACL    */
#    IS 0.7 FROM VOLUME OF MIXING CALCULATIONS                       */

h = eV.iloc[2, 0]  # Adjusted index to match the DataFrame
h = float(h.strip())

na = 0.7 - h
cl = 0.7

#CALCULATE THE IONIC STRENGTH TERMS									*/

I0 = (na + cl + h) / 2

#CALCULATE THE TOTAL EQUIVALENTS AND EQUIVALENT FRACTIONS			*/

E = (na + cl + h) / 2
na = na / E
cl = cl / E
h = h / E
M = na * 22.9898 + cl * 35.453 + h * 1.008


#HCL FROM DENSITY EQUATIONS											
hcl = (20.3368 - 0.0737834 * t - 5.29257e-3 * t * t + 4.50398e-4 * t * t * t - 1.17417e-5 * t * t * t * t + 1.02433e-7 * t * t * t * t * t) * I - 1.46902 * I * np.sqrt(I)
hcl = hcl / 1000 + d0  # density g/cm3
phihcl = 1000 * (d0 - hcl) / (I * hcl * d0) + (1.008 + 35.453) / hcl

#	SODIUM SALTS FROM DENSITY EQUATIONS		
nacl = (45.5655 - .2341 * t + 0.0034128 * t * t - 2.703e-5 * t * t * t + 1.4037e-7 * t * t * t * t) * I + (-1.8527 + 0.053956 * t - 6.2635e-4 * t * t) * I * np.sqrt(I) + (-1.6368 - 9.5653e-4 * t + 5.2829e-5 * t * t) * I * I + 0.2274 * I * I * np.sqrt(I)
nacl = (nacl / 1000) + d0
phinacl = 1000 * (d0 - nacl) / (I * nacl * d0) + (22.9898 + 35.453) / nacl


#PHIV OF THE MIXTURE				*/
pna = phinacl * na * cl
ph = h * cl * phihcl
phimix = pna + ph

rho_hcl = (M * E + 1000) * d0 * (1 / (phimix * E * d0 + 1000))

cellV = eV.iloc[0, 1] / rho_sw  # Adjusted index to match the DataFrame
E0 = cellV / 1000
observed = []
for i in range(len(mV)):
    obs = np.exp(-((E0 - mV[0:i]) / T_NS) * np.log(10))
    observed.append(obs)
#observed = np.array(observed)

observed





[2    0.0
 Name: mV, dtype: float64,
 2    0.0
 3    0.0
 Name: mV, dtype: float64,
 2    0.0
 3    0.0
 4    0.0
 Name: mV, dtype: float64,
 2    0.0
 3    0.0
 4    0.0
 5    0.0
 Name: mV, dtype: float64,
 2    0.0
 3    0.0
 4    0.0
 5    0.0
 6    0.0
 Name: mV, dtype: float64,
 2    0.0
 3    0.0
 4    0.0
 5    0.0
 6    0.0
 7    0.0
 Name: mV, dtype: float64,
 2    0.0
 3    0.0
 4    0.0
 5    0.0
 6    0.0
 7    0.0
 8    0.0
 Name: mV, dtype: float64,
 2    0.0
 3    0.0
 4    0.0
 5    0.0
 6    0.0
 7    0.0
 8    0.0
 9    0.0
 Name: mV, dtype: float64,
 2     0.0
 3     0.0
 4     0.0
 5     0.0
 6     0.0
 7     0.0
 8     0.0
 9     0.0
 10    0.0
 Name: mV, dtype: float64,
 2     0.0
 3     0.0
 4     0.0
 5     0.0
 6     0.0
 7     0.0
 8     0.0
 9     0.0
 10    0.0
 11    0.0
 Name: mV, dtype: float64,
 2     0.0
 3     0.0
 4     0.0
 5     0.0
 6     0.0
 7     0.0
 8     0.0
 9     0.0
 10    0.0
 11    0.0
 12    0.0
 Name: mV, dtype: float64,
 2     0.0
 3

In [ ]:
df = pd.DataFrame(vol, mV)
sig = []

cellE0
for i in df
    vol[i]=vol[i]*(rho_hcl)
    mV[i]=mV[i]/1000
    sig[i]=.1/1000
    E0=cellE0/1000


Bt, St, kb, kw, k1, k2
Z = 1 + (St/ks)

E0 =   # Initial estimate of E0
emf = mV / 1000  # Convert mV to V

hp = np.exp(-((E0 - emf) / T_NS) * np.log(10))
f = h/hp

Carb = DIC*(k1*f*hp + 2*k1*k2)/(f*hp**2+k1*f*hp+k1*k2)
Boron = Bt* (1/(1+(f*hp/kb)))
Sulf = St*(1/(1+(ks*Z)/f*hp))




# Define the model function
def model(E0, mV, T_NS):
    emf = mV / 1000  # Convert mV to V
    hp = np.exp(-((E0 - emf) / T_NS) * np.log(10))
    f = h / hp
    Carb = DIC * (k1 * f * hp + 2 * k1 * k2) / (f * hp**2 + k1 * f * hp + k1 * k2)
    Boron = Bt * (1 / (1 + (f * hp / kb)))
    Sulf = St * (1 / (1 + (ks * Z) / (f * hp)))
    mass = (m0 + m) / m0
    etc = (f * hp) / Z - (kw / (f * hp))
    return Carb - Boron + Sulf + mass * etc - (m / m0) * h

fit_func = Carb - Boron + Sulf + mass*etc - (m/m0)*h

fit = lm.minimize(fit_func)

# Define the objective function to minimize
def objective(params, mV, T_NS):
    E0 = params['E0']
    return model(E0, mV, T_NS)

# Create a set of Parameters
params = lm.Parameters()
params.add('E0', value=0.1)  # Initial guess for E0

# Perform the minimization
result = lm.minimize(objective, params, args=(mV, T_NS))

# Print the results
lm.report_fit(result)
mass = (m0 + m)/m0
etc = (f*hp)/Z - (kw/f*hp)


np.float64(1.0250080592362012)

In [106]:


from lmfit import minimize, Parameters

def fitted_ht(params, mV, vol, consts):
    """
    Calculate the H+ concentration using an iterative Newton's method.
    """
    H = []
    pHtol = 1e-5  # Convergence tolerance
    for i, mV_i in enumerate(mV):
        # Extract parameters
        af1 = params['af1'].value
        af2 = params['af2'].value
        af3 = params['af3'].value
        af4 = params['af4'].value
        af5 = params['af5'].value
        
        # Compute initial guess for H
        Z = 1 + (St/ks)

        E0 = 426  # Initial estimate of E0
       # emf = mV[i] / 1000  # Convert mV to V
        H_i = np.exp(-((E0 - mV[0]) / T_NS) * np.log(10))  # Starting value

        # Newton's method to refine H_i
        mmm = 0
        while mmm < 100:
            ctmul = (af4 * H_i + 2 * af4 * af5) / (H_i ** 2 + af4 * H_i + af4 * af5)
            btmul = 1 / (1 + H_i / kb)
            stmul = 1 / (1 + ks *Z / H_i)

            hmul = (cellV + vol[i]) / cellV
            zero = (
                af2
                - af3 * ctmul
                - Bt * btmul
                + St * stmul
                + hmul * (H_i / Z - kw / H_i)
                - vol[i] / cellV * h
            )
            
            slope = (
                np.log(10)
                * (
                    af3 * af4 * H_i
                    * (H_i ** 2 + af4 * af5 + 4 * H_i * af5)
                    / (H_i ** 2 + af4 * H_i + af4 * af5) ** 2
                    + Bt * btmul * H_i / (kb + H_i)
                    + 1 / hmul * consts[9] / H_i
                    + hmul * H_i
                )
            )
            
            deltapH = zero / slope
            if abs(deltapH) > 1:
                deltapH /= 2
            if abs(deltapH) < pHtol:
                break
            H_i *= 10 ** -deltapH
            mmm += 1
        
        H.append(H_i if H_i > 0 else -1)
    
    return np.array(H)

def residual(params, mV, vol, consts, observed):
    """
    Compute residuals between observed and modeled H+ concentrations.
    """
    modeled = fitted_ht(params, mV, vol, consts)
    return observed - modeled

# Define parameters
params = Parameters()
params.add('af1', value=0.01, min=0)
params.add('af2', value=0.01, min=0)
params.add('af3', value=0.01, min=0)
params.add('af4', value=0.01, min=0)
params.add('af5', value=0.01, min=0)

# Example data
#emf = np.array([0.2, 0.25, 0.3])  # Example emf values
  # Example volume values
consts = np.array([kb,ks,kw,Bt,St])  # Replace with actual constants
#observed = np.array([0.01, 0.015, 0.02])  # Observed H+ concentrations

# Minimize residuals
result = minimize(residual, params, args=(mV, vol, consts, observed))

# Print results
print(result.params)


KeyError: 0